<a href="https://colab.research.google.com/github/keipertk/pygpu-workshop/blob/master/pygpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites

This notebook presents libraries and tools for making Python code run on
NVIDIA GPUs!! The path to running on GPUs goes through improving CPU
performance but the target is to run on GPUs.

The notebook was tested on the Anaconda Python distribution for Python 3.x.
This container should have all of the package dependencies.

This notebook follows the accompanying slides.

---

Let's check which CPU and GPU you're using. 

In [1]:
!nvidia-smi

Fri Feb 26 14:57:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

We can write Python and execute it inside a code block: 

In [3]:
print("Hello world!!!!!")

Hello world!!!!!


<a id='Simple Python Example with Numba'></a>
# 1. Simple Python Example Using Numba - CPU and GPU


This is a simple example that adds two vectors to create a third vector.
The computational work is done in a function. To create enough computational
work, the vector length is quite large (1,000,000,000).
These are single precision vectors.

The example starts with a simple Python example with no compiling.
Just in case you are interested a simple timing of the addition is
printed at the end.

<a id='Simple Python Example'></a>
## 1.1 Starting Point - Simple Python Code

The code below is the baseline Python code (the starting point). It simply
adds the elements of two vectors together and stores that in a third vector.
The arrays are created using NumPy. The time it takes to perform the addition
is measured using the "time" module in Python.

Based on code from: https://devblogs.nvidia.com/numba-python-cuda-acceleration/ 

In [10]:
import numpy as np
from time import perf_counter

def Add(a, b):
    return a + b
# end def

# Initialize arrays
N = 1000000000
A = np.ones(N, dtype=np.float32)
B = np.ones(A.shape, dtype=A.dtype)
C = np.empty_like(A, dtype=A.dtype)

# Add arrays on CPU
start_time = perf_counter()
C = Add(A, B)
stop_time = perf_counter()

print(C)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )

del A
del B
del C

[2. 2. 2. ... 2. 2. 2.]

    Elapsed wall clock time = 1.17532 seconds.


<a id='Simple Python Example - CPU'></a>
## 1.2 Simple Python - CPU Examples

The following examples present code that uses Numba to compile the <tt>Add</tt>
function in the code. Various options are presented for both CPU compiling
and compiling for the NVIDIA GPU. These options use various decorators and
options. The discussion to accompany these examples are in the slide deck.

<a id='Simple Python Example single core'></a>
### 1.2.1 <tt>@jit</tt> with default target (single CPU core)

This example uses the <tt>@jit</tt> decorator in Numba. By default this targets a
single core on the CPU.

You might try running the cell a few times to get an idea of the average
wall clock time.

In [13]:
#@title
import numpy as np
from time import perf_counter
from numba import jit

@jit
def Add(a, b):
    return a + b
# end def


# Initialize arrays
N = 1000000000
A = np.ones(N, dtype=np.float32)
B = np.ones(A.shape, dtype=A.dtype)
C = np.empty_like(A, dtype=A.dtype)

# Add arrays on CPU
start_time = perf_counter( )
C = Add(A, B)
stop_time = perf_counter( )

print(C)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

del A
del B
del C

[2. 2. 2. ... 2. 2. 2.]

    Elapsed wall clock time = 1.42924 seconds.



<a id='Simple Python Example multi-core'></a>
### 1.2.2 <tt>@jit</tt> with multi-core target and no Python fallback

This targets multiple CPU cores using the  <tt>parallel=True</tt>  option in  <tt>@jit</tt>.

I also use the option  <tt>nopython=True</tt>  so I can catch my code
mistakes. It is a default option, but I like to specify so I know exactly
what options I'm using. This option disables any Python fall-back in case
Numba cannot compile the code.

You might try running the cell a few times to get an idea of the average
wall clock time.

In [18]:
#@title
import numpy as np
from time import perf_counter
from numba import jit

@jit(nopython=True, parallel=True) 
def Add(a, b):
    return a + b
# end def

# Initialize arrays
N = 1000000000
A = np.ones(N, dtype=np.float32)
B = np.ones(A.shape, dtype=A.dtype)
C = np.empty_like(A, dtype=A.dtype)

# Add arrays on CPU
start_time = perf_counter( )
C = Add(A, B)
stop_time = perf_counter( )

print(C)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

del A
del B
del C

[2. 2. 2. ... 2. 2. 2.]

    Elapsed wall clock time = 1.45008 seconds.



<strong>Note</strong>: Currently, we only measure the time for the compiled
code. We don't measure
the time that includes the compile time. Therefore, the elapsed wall clock
times should not vary too much.

<a id='Simple Python Example defaults cache'></a>
### 1.2.3 <tt>@jit</tt> with defaults, caching, and no Python fallback

This example uses the defaults again, but it also caches the compiled code
so it can be reused (<tt>cache=True</tt>). 

In [19]:
#@title
import numpy as np
from time import perf_counter
from numba import jit

@jit(nopython=True, cache=True) 
def Add(a, b):
    return a + b
# end def

# Initialize arrays
N = 1000000000
A = np.ones(N, dtype=np.float32)
B = np.ones(A.shape, dtype=A.dtype)
C = np.empty_like(A, dtype=A.dtype)

# Add arrays on CPU
start_time = perf_counter( )
C = Add(A, B)
stop_time = perf_counter( )

print(C)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

del A
del B
del C

[2. 2. 2. ... 2. 2. 2.]

    Elapsed wall clock time = 1.40199 seconds.



<a id='Simple Python Example multicore cache'></a>
### 1.2.4 <tt>@jit</tt> with parallel, caching, and no Python fallback

This example adds the option <tt>parallel=True</tt> (multi-core) to the
previous options.

In [ ]:
#@title
import numpy as np
from time import perf_counter
from numba import jit

@jit(nopython=True, parallel=True, cache=True) 
def Add(a, b):
    return a + b
# end def

# Initialize arrays
N = 1000000000
A = np.ones(N, dtype=np.float32)
B = np.ones(A.shape, dtype=A.dtype)
C = np.empty_like(A, dtype=A.dtype)

# Add arrays on CPU
start_time = perf_counter( )
C = Add(A, B)
stop_time = perf_counter( )

print(C)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

[2. 2. 2. ... 2. 2. 2.]

    Elapsed wall clock time = 1.39953 seconds.



<a id='Simple Python Example vectorize no type signature single core'></a>
### 1.2.5 <tt>@vectorize</tt> decorator for CPUs (no type signature)

This example uses the @vectorize decorator targeting the CPU (default target). 
Remember that the <tt>@vectorize</tt> decorator compiles functions that perform
element-by-element operations (that is, the same operation to every element
in the array). The code will most likely contain loops.

In general, you give the <tt>@vectorize</tt> decorator a type signature that
tells Numba how to build the compiled code for a specific data type (you can
specify more than one which is really cool). Here is an example,


<codeblock>
    @vectorize(['float32(float32, float32)'])
</codeblock>


The data type specification after the decorator is the "type signature".

You don't have to specify a type signature. If you don't, then Numba will
create a dynamic universal function (DUFunc). This dynamically compilers a
new kernel when the function is called with a data type that wasn't previously
used. That is, it will compile the function for every specific data type in
your code. This can have an impact if your use multiple data types with the same
function. You can think of this approach as "call-time" or "lazy" compilation.

Below is an example of using the <tt>@vectorize</tt> decorator without a type
signature.

In [ ]:
#@title
import numpy as np
from time import perf_counter
from numba import vectorize

@vectorize
def Add(a, b):
    return a + b
# end def

# Initialize arrays
N = 1000000000
A = np.ones(N, dtype=np.float32)
B = np.ones(A.shape, dtype=A.dtype)
C = np.empty_like(A, dtype=A.dtype)

# Add arrays on CPU
start_time = perf_counter( )
C = Add(A, B)
stop_time = perf_counter( )

print(C)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

del A
del B
del C

[2. 2. 2. ... 2. 2. 2.]

    Elapsed wall clock time = 1.95581 seconds.



<a id='Simple Python Example vectorize type signature single core'></a>
### 1.2.6 @vectorize decorator for CPUs (type signature)

Below is an example of using the @vectorize decorator with "type signature(s)".
Only a single type signature is defined, but you can
provide several "type signatures", each with different data types if you
wished (notice that the type signature is an list (array) ). You would probably
only do this if you used the same function with different data types.

In [ ]:
#@title
import numpy as np
from time import perf_counter
from numba import vectorize

@vectorize(['float32(float32, float32)'])
def Add(a, b):
    return a + b
# end def

# Initialize arrays
N = 1000000000
A = np.ones(N, dtype=np.float32)
B = np.ones(A.shape, dtype=A.dtype)
C = np.empty_like(A, dtype=A.dtype)

# Add arrays on CPU
start_time = perf_counter( )
C = Add(A, B)
stop_time = perf_counter( )

print(C)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

del A
del B
del C

[2. 2. 2. ... 2. 2. 2.]

    Elapsed wall clock time = 0.994248 seconds.



<a id='Simple Python Example vectorize type signature single core'></a>
### 1.2.7 @vectorize decorator for CPUs with single CPU core target

This is a trivial example, but it illustrates that the default target for the
<tt>@vectorize</tt> decorator is a single core ( <tt>target='cpu'</tt> ). 

In [ ]:
#@title
import numpy as np
from time import perf_counter
from numba import vectorize

@vectorize(['float32(float32, float32)'], target='parallel')
def Add(a, b):
    return a + b
# end def

# Initialize arrays
N = 1000000000
A = np.ones(N, dtype=np.float32)
B = np.ones(A.shape, dtype=A.dtype)
C = np.empty_like(A, dtype=A.dtype)

# Add arrays on CPU
start_time = perf_counter( )
C = Add(A, B)
stop_time = perf_counter( )

print(C)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

del A
del B
del C

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


[2. 2. 2. ... 2. 2. 2.]

    Elapsed wall clock time = 12.0903 seconds.



<a id='Simple Python Example vectorize type signature parallel'></a>
### 1.2.8 @vectorize decorator using all CPU cores

This example changes the target to use all of the cores (multi-core). The target name
is simple <tt>parallel</tt>. Even though all of the cores are being used, the performance
may not improve. It takes time to move data around as needed across the cores.

In [ ]:
#@title
import numpy as np
from time import perf_counter
from numba import vectorize

@vectorize(['float32(float32, float32)'], target='parallel')
def Add(a, b):
    return a + b
# end def

# Initialize arrays
N = 1000000000
A = np.ones(N, dtype=np.float32)
B = np.ones(A.shape, dtype=A.dtype)
C = np.empty_like(A, dtype=A.dtype)

# Add arrays on CPU
start_time = perf_counter( )
C = Add(A, B)
stop_time = perf_counter( )

print(C)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

del A
del B
del C

[2. 2. 2. ... 2. 2. 2.]

    Elapsed wall clock time = 11.1745 seconds.



<a id='Simple Python Example loops'></a>
### 1.2.9 Using @jit with loops in the function

The simple Python code uses array functions. What should you do if the code
uses loops? The next cell illustrate how to write the appropriate
Python code for Numba. It also shows how to use the  <tt>@jit</tt>  decorator
on CPUs.

The first cell uses the  <tt>@jit</tt>  decorator.

In [ ]:
#@title
# Using the @jit decorator

import numpy as np
from time import perf_counter
from numba import jit

@jit
def Add(a, b, c):
    max_length = len(a)
    for i in range(0, max_length):
        c[i] = a[i] + b[i]
    # end for
# end def

# Initialize arrays
N = 1000000000
A = np.ones(N, dtype=np.float32)
B = np.ones(A.shape, dtype=A.dtype)
C = np.empty_like(A, dtype=A.dtype)

# Add arrays on CPU
start_time = perf_counter( )
Add(A, B, C)
stop_time = perf_counter( )

print(C)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

del A
del B
del C

[2. 2. 2. ... 2. 2. 2.]

    Elapsed wall clock time = 1.34306 seconds.



<a id='Simple Python Example - GPU'></a>
## 1.3 Simple Python - GPU Examples

The examples below explore using the GPU as a target with the vectorize decorator
and the <tt>cuda</tt> target, as well as using the  <tt>@cuda.jit</tt>  decorator.

<a id='Simple Python Example - Vectorize GPU'></a>
### 1.3.1 <tt>@vectorize</tt> Decorator with a CUDA Target

Porting Python code to the GPU can be very easy using the code from the  <tt>@vectorize</tt>
decorator. For most code, all you need to do is change the *target* in the decorator to
<tt>cuda</tt>.

Notice how Numba takes care of copying the data to the GPU and copying it back. Numba
also takes care of defining the arrays on the GPU.

Try running the cell several times to get an idea of the compute time.

In [25]:
import numpy as np
from time import perf_counter
from numba import vectorize

@vectorize(['float32(float32, float32)'], target='cuda')
def Add(a, b):
    return a + b
# end def

# Initialize arrays
N = 1000000000
A = np.ones(N, dtype=np.float32)
B = np.ones(A.shape, dtype=A.dtype)
C = np.empty_like(A, dtype=A.dtype)

# Add arrays on GPU
start_time = perf_counter( )
C = Add(A, B)
stop_time = perf_counter( )

print(C)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

del A
del B
del C


[2. 2. 2. ... 2. 2. 2.]

    Elapsed wall clock time = 3.0923 seconds.



<a id='Simple Python Example - cud.jit GPU'></a>
### 1.3.2 <tt>@cuda.jit</tt> Decorator

You can also use a different decorator,  <tt>@cuda.jit</tt>, that allows you to
write Python code that is more CUDA-like. This offers
you greater flexibility and control over your code and possibly better performance.
However, you will need to know a fair amount about CUDA  before using it.
The link below is a good place to get more information.


http://numba.pydata.org/numba-doc/dev/cuda/index.html

    
As explained in the slides, using the  <tt>cuda.jit</tt>  decorator requires some extra coding.
You have to explicitly write the loops in your code.
Also, everything you pass into or out-of the compiled function, has to be a NumPy
array (even scalars which are NumPy arrays of size 1). However, you can define simple
scalars in the function that are not NumPy arrays. For example, loop counters.

Pay close attention to how the data is passed to the function - it is much more
like functions in C where everything all data is passed through the function call. 

To prepare for using <tt>@cuda.jit</tt> the cell below rewrites the Python code into
loops and uses the <tt>@cuda.jit<tt> decorator.

In [26]:
import numpy as np
from time import perf_counter
from numba import cuda

@cuda.jit
def Add(a, b, c):
    max_length = len(a)
    for i in range(0, max_length):
        c[i] = a[i] + b[i]
    # end for
# end def

# Initialize arrays
N = 1000000000
A = np.ones(N, dtype=np.float32)
B = np.ones(A.shape, dtype=A.dtype)
C = np.empty_like(A, dtype=A.dtype)

# Add arrays on GPU
start_time = perf_counter( )
Add[32,32](A, B, C)
stop_time = perf_counter( )

print(C)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

del A
del B
del C

[2. 2. 2. ... 2. 2. 2.]

    Elapsed wall clock time = 89.3798 seconds.



<a id='Python Porting Example'></a>
# 2. Python Porting Example

This is an example of porting functions to use Numba. It is really an example of the porting
<em>lifecycle</em> of a Python application. It starts with a serial Python code
that has been written to test the idea and to make sure it works and gets correct answers.

Then it moves to putting computational intensive portions of the code into functions.

Then it uses Numba to compile for the CPU, using the <tt>@jit</tt> decorator. 

The next step is to switch to the <tt>@vectorize</tt> decorator, targeting a single core
(target is <tt>cpu</tt>), then all of the CPU cores (target is <tt>parallel</tt>), and
finally to NVIDIA GPUs (target is <tt>cuda</tt>).

The last step is to modify the code to use the <tt>@cuda.jit</tt> decorator. This
requires some code changes.

The new example code is a very, very simplified version of the start of a Molecular
Dynamic (MD) mini-app. It focuses on loops that are initialized (the values are arbitrary
and don't mean anything). The code an obviously be made much simpler and faster, but that is
not the point. The point is to show you how start with a code that uses loops and "port"
it to use Numba and GPUs. Several steps will be used in this porting process. Hopefully it
gives you some "feel" in porting your applications to use Numba.
                                                    
Let's start with the serial Python code.

In [ ]:
import numpy as np
from time import perf_counter

# main loop
start_time = perf_counter( )
d_num = 5000
p_num = 5000

pos = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
accel = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
for j in range(0, p_num):
    for i in range(0, d_num):
        pos[i,j] = 6.5
    # end for i
    for i in range(0, d_num):
        accel[i,j] = 4.2*pos[i,j]
    # end for
# end for j

stop_time = perf_counter( )

print(pos)
print(accel)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

[[6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 ...
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]]
[[27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 ...
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]]

    Elapsed wall clock time = 57.6653 seconds.



<a id='Non-Numba Python Code - Function'></a>
## 2.1 Non-Numba Python Code

The next code creates a function for the computationally intense part of the code (the loop). 
This gets the code ready for Numba (Remember that Numba compiles functions, not entire codes).
Notice that the two arrays are created in the function and are returned to the calling function
(both arrays are returned)

In [ ]:
import numpy as np
from time import perf_counter

def init(p_num, d_num):
    pos = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
    accel = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
    for j in range(0, p_num):
        for i in range(0, d_num):
            pos[i,j] = 6.5
        # end for i
        for i in range(0, d_num):
            accel[i,j] = 4.2*pos[i,j]
        # end for i
    # end for j
    return pos, accel
# end def


# main
d_num = 5000
p_num = 5000

start_time = perf_counter( )
pos, accel = init(p_num, d_num)
stop_time = perf_counter( )

print(pos)
print(accel)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

Always be sure to check that you answers are correct after you take another
step in porting it. Checking answers for this simple code is fairly easy
and can be done by simply printing the arrays.

<a id='Python Porting Example jit decorator single'></a>
## 2.2 <tt>@jit</tt> Decorator Targeting a Single Core on the CPU

Next, let us use the <tt>@jit</tt> decorator to compile the function. The first one uses the
default target which is a single core.

Note: If you want to eliminate the time to compile, run the cell again without changing the code.
You can do this several times to get an understanding of the true run time without including
the compilation time.

In [ ]:
import numpy as np
from time import perf_counter
from numba import jit

@jit
def init(p_num, d_num):
    pos = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
    accel = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
    for j in range(0, p_num):
        for i in range(0, d_num):
            pos[i,j] = 6.5
        # end for i
        for i in range(0, d_num):
            accel[i,j] = 4.2*pos[i,j]
        # end for i
    # end for j
    return pos, accel
# end def


# main
d_num = 5000
p_num = 5000

start_time = perf_counter( )
pos, accel = init(p_num, d_num)
stop_time = perf_counter( )

print(pos)
print(accel)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

[[6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 ...
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]]
[[27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 ...
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]]

    Elapsed wall clock time = 0.966766 seconds.



<a id='Python Porting Example jit decorator parallel'></a>
## 2.3 <tt>@jit</tt> Decorator Targeting Multi-Core (parallel=True)

This code simply compiles for multi-core and also disables Python fallback.

Note: If you want to eliminate the time to compile, run the cell again without changing the code.
You can do this several times to get an understanding of the true run time without including
the compilation time.

In [ ]:
import numpy as np
from time import clock
from numba import jit

@jit(nopython=True, parallel=True)
def init(p_num, d_num):
    pos = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
    accel = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
    for j in range(0, p_num):
        for i in range(0, d_num):
            pos[i,j] = 6.5
        # end for i
        for i in range(0, d_num):
            accel[i,j] = 4.2*pos[i,j]
        # end for i
    # end for j
    return pos, accel
# end def


# main
d_num = 5000
p_num = 5000

start_time = perf_counter( )
pos, accel = init(p_num, d_num)
stop_time = perf_counter( )

print(pos)
print(accel)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

[[6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 ...
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]]
[[27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 ...
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]]

    Elapsed wall clock time = 1.08775 seconds.



<a id='Python Porting Example vectorize single'></a>
## 2.4 <tt>@vectorize</tt> Decorator for a Single Core (default)

This example rewrites the code to use the <tt>@vectorize</tt> decorator. Remember that this
requires the code to be executed element-by-element. It really also requires that
each function return one result. This will require some code changes to split the
two loops so that each has its own function.

Recall that we write the function as if it were a scalar. Numba takes care of
all of the details of creating the ufunc based on the code.

Another thing to note is that we have to create the arrays in the main routine. They
can no longer be created in the functions. The <tt>@vectorize</tt> decorator only wants
very simple element-by-element code. The precludes creating the arrays in the functions.

Pay close attention to the type signature(s) for the decorator and how the functions
are called. It is a bit counter intuitive. 

Note: If you want to eliminate the time to compile, run the cell again without changing the code.
You can do this several times to get an understanding of the true run time without including
the compilation time.

In [ ]:
import numpy as np
from time import perf_counter
from numba import vectorize

@vectorize(['float32(float32)'])
def set_pos(pos):
    return 6.5
# end def

@vectorize(['float32(float32, float32)'])
def set_accel(pos, accel):
    return 4.2*pos
# end def


# main
d_num = 5000
p_num = 5000
pos = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
accel = np.zeros( shape=(d_num, p_num), dtype=np.float32 )

start_time = perf_counter( )
pos = set_pos(pos)
accel = set_accel(pos,accel)
stop_time = perf_counter( )

print(pos)
print(accel)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

[[6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 ...
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]]
[[27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 ...
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]]

    Elapsed wall clock time = 0.0432454 seconds.



<a id='Python Porting Example vectorize parallel'></a>
## 2.5 <tt>@vectorize</tt> Decorator for Multi-Core

This example simply changes the target for the <tt>@vectorize</tt> decorator to multi-core (parallel).

Note: If you want to eliminate the time to compile, run the cell again without changing the code.
You can do this several times to get an understanding of the true run time without including
the compilation time.

In [ ]:
import numpy as np
from time import perf_counter
from numba import vectorize

@vectorize(['float32(float32)'], target='parallel')
def set_pos(pos):
    return 6.5
# end def

@vectorize(['float32(float32, float32)'], target='parallel')
def set_accel(pos, accel):
    return 4.2*pos
# end def


# main
d_num = 10000
p_num = 10000
pos = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
accel = np.zeros( shape=(d_num, p_num), dtype=np.float32 )

start_time = perf_counter( )
pos = set_pos(pos)
accel = set_accel(pos,accel)
stop_time = perf_counter( )

print(pos)
print(accel)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

[[6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 ...
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]]
[[27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 ...
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]]

    Elapsed wall clock time = 2.20153 seconds.



<a id='Python Porting Example vectorize cuda'></a>
## 2.6 <tt>@vectorize</tt> Decorator - CUDA Target

This next example also simply changes the target, but this time it is for NVIDIA GPUs.
THis is a benefit of being able to write your function code as scalars and using Numba
to vectorize it. You can just change the target to get either a single CPU core,
multiple CPU cores, or an NVIDIA GPU.

Note: If you want to eliminate the time to compile, run the cell again without changing the code.
You can do this several times to get an understanding of the true run time without including
the compilation time.

In [ ]:
import numpy as np
from time import perf_counter
from numba import vectorize

@vectorize(['float32(float32)'], target='cuda')
def set_pos(pos):
    return 6.5
# end def

@vectorize(['float32(float32, float32)'], target='cuda')
def set_accel(pos, accel):
    return 4.2*pos
# end def


# main
d_num = 10000
p_num = 10000
pos = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
accel = np.zeros( shape=(d_num, p_num), dtype=np.float32 )

start_time = perf_counter( )
pos = set_pos(pos)
accel = set_accel(pos,accel)
stop_time = perf_counter( )

print(pos)
print(accel)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

[[6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 ...
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]]
[[27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 ...
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]]

    Elapsed wall clock time = 0.78537 seconds.



<a id='Python Porting Example cuda.jit'></a>
## 2.7 <tt>@cuda.jit</tt> Decorator for NVIDIA GPUs

To use the <tt>@cuda.jit</tt> decorator, we will need to rewrite the function code. Remember
that we have to write the explicit loops for the code now (it's not element-by-element).
We have to change a few other things listed below.

1. Everything is a Numpy array (even scalars - Numpy array of size 1)
2. Cannot define new arrays in function and copy to host. Need to define them on host first, then copy them to device ( <tt>cuda.to_device()</tt> )
3. Refer to scalars as <tt>array[0]</tt> (first element of array). You could create a "local" scalar in the functions if needed
4. No values are returned using <tt>return</tt> function. They have to be copied back using <tt>copy_to_host</tt> method.
5. Variables stay on GPU unless you explicitly delete them (opportunity here)
6. Copying data between host/device can waste time - limit the amount of data transfer

Notice that the timings include data transfer times. You can change this if you
want to only time the computational portion that is on the GPU.

Note: If you want to eliminate the time to compile, run the cell again without changing the code.
You can do this several times to get an understanding of the true run time without including
the compilation time.

In [ ]:
import numpy as np
from time import perf_counter
from numba import cuda

@cuda.jit
def init(p_num, d_num, pos, accel):
    for j in range(0, p_num[0]):
        for i in range(0, d_num[0]):
            pos[i,j] = 6.5
        # end for i
        for i in range(0, d_num[0]):
            accel[i,j] = 4.2*pos[i,j]
        # end for i
    # end for j
# end def


# main loop
d_num = np.zeros(1, dtype=int)
p_num = np.zeros(1, dtype=int)
d_num[0] = 500
p_num[0] = 500
pos = np.zeros( shape=(d_num[0], p_num[0]), dtype=np.float32 )
accel = np.zeros( shape=(d_num[0], p_num[0]), dtype=np.float32 )

start_time = perf_counter( )

#By default, any NumPy arrays used as argument of a CUDA kernel
# is transferred automatically to and from the device

#d_p_num = cuda.to_device(p_num)
#d_d_num = cuda.to_device(d_num)
#d_pos = cuda.to_device(pos)
#d_accel = cuda.to_device(accel)

init[32,32](p_num, d_num, pos, accel)

stop_time = perf_counter( )

print(pos)
print(accel)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

[[6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 ...
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]]
[[27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 ...
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]]

    Elapsed wall clock time = 0.378787 seconds.



<a id='Functions Calling Functions'></a>
# 3. Functions Calling Functions

When running on the CPU, it's easy for a JIT compiled routine to call another JIT
compiled routine or even just another Python routine. There is no data movement
since there is only one pool of memory. However, calling JIT compiled GPU functions
from other JIT compiled GPU functions is a little bit different.

To explore how calling compiled functions from other compiled functions, let's
begin with a modification of our serial Python code, adding a new function.

In [ ]:
import numpy as np
from time import perf_counter
from numba import jit

def init_pos_accel(p_num, d_num):
    pos = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
    accel = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
    for j in range(0, p_num):
        for i in range(0, d_num):
            pos[i,j] = 6.5
        # end for i
        for i in range(0, d_num):
            accel[i,j] = 4.2*pos[i,j]
        # end for i
     # end for j
    
    vel = init_vel(p_num, d_num, pos, accel)

    return pos, accel, vel
# end init

def init_vel(p_num, d_num, pos, accel):
    vel = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
    for j in range(0, p_num):
        for i in range(0, d_num):
            vel[i,j] = pos[i,i] + accel[i,j] * 0.1*pos[i,j]
        # end for i
    # end for j

    return vel
# end def


# main
d_num = 5000
p_num = 5000

start_time = perf_counter( )
pos, accel, vel = init_pos_accel(p_num, d_num)
stop_time = perf_counter( )

print(pos)
print(vel)
print(accel)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

[[6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 ...
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]]
[[24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 ...
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]]
[[27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 ...
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]]

    Elapsed wall clock time = 176.557 seconds.



<a id='Functions Calling Functions jit'></a>
## 3.1 Functions Calling Functions - CPUs with <tt>@jit</tt>

For the a CPU example, let's compile both functions for the CPU using the
<tt>@jit</tt> decorator. You just put the decorator before each function
declaration.

In [ ]:
import numpy as np
from time import perf_counter
from numba import jit

@jit
def init_pos_accel(p_num, d_num):
    pos = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
    accel = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
    for j in range(0, p_num):
        for i in range(0, d_num):
            pos[i,j] = 6.5
        # end for i
        for i in range(0, d_num):
            accel[i,j] = 4.2*pos[i,j]
        # end for i
    # end for j
    
    vel = init_vel(p_num, d_num, pos, accel)

    return pos, accel, vel
# end def

@jit
def init_vel(p_num, d_num, pos, accel):
    vel = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
    for j in range(0, p_num):
        for i in range(0, d_num):
            vel[i,j] = pos[i,i] + accel[i,j] * 0.1*pos[i,j]
        # end for i
    # end for j

    return vel
# end def


# main
d_num = 5000
p_num = 5000

start_time = perf_counter( )
pos, accel, vel = init_pos_accel(p_num, d_num)
stop_time = perf_counter( )

print(pos)
print(accel)
print(vel)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

[[6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 ...
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]]
[[27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 ...
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]]
[[24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 ...
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]]

    Elapsed wall clock time = 2.23552 seconds.



<a id='Functions Calling Functions jit Parallel'></a>
## 3.2 Functions Calling Functions - <tt>@jit</tt> Targeting Multi-Core CPU

For fun, let's compile the previous code but target multi-core CPUs.

In [ ]:
import numpy as np
from time import perf_counter
from numba import jit

@jit(nopython=True, parallel=True)
def init_pos_accel(p_num, d_num):
    pos = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
    accel = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
    for j in range(0, p_num):
        for i in range(0, d_num):
            pos[i,j] = 6.5
        # end for i
        for i in range(0, d_num):
            accel[i,j] = 4.2*pos[i,j]
        # end for i
    # end for j
    
    vel = init_vel(p_num, d_num, pos, accel)

    return pos, accel, vel
# end def

@jit(nopython=True, parallel=True)
def init_vel(p_num, d_num, pos, accel):
    vel = np.zeros( shape=(d_num, p_num), dtype=np.float32 )
    for j in range(0, p_num):
        for i in range(0, d_num):
            vel[i,j] = pos[i,i] + accel[i,j] * 0.1*pos[i,j]
        # end for i
    # end for j

    return vel
# end def


# main
d_num = 5000
p_num = 5000

start_time = perf_counter( )
pos, accel, vel = init_pos_accel(p_num, d_num)
stop_time = perf_counter( )

print(pos)
print(accel)
print(vel)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

[[6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 ...
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]]
[[27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 ...
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]]
[[24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 ...
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]]

    Elapsed wall clock time = 3.63042 seconds.



<a id='Functions Calling Functions cuda.jit'></a>
## 3.3 Functions calling Functions - GPUs with <tt>@cuda.jit</tt>

A simple way for a JIT compiled GPU function to call another JIT compiled
GPU function is to put the code for all GPU functions into
a single function (kind of like stuffing your whole code into a function).
This can make things complicated and may also confuse the compiler causing
it to produce slower code.

Fortunately, <tt>@cuda.jit</tt> provides a reasonable way to compile multiple functions.
The key is to tell the functions _after_ the first one that the data is already on the
device. This is done through an option with the decorator.


<codeblock>
@cuda.jit(device=True)
</codeblock>


Don't forget that when using the <tt>@cuda.jit</tt> decorator that you need to make
everything a numpy array, even scalars. The code below illustrates this and also
includes the decorators with the appropriate option.



<strong>References</strong>:
    
1. <a href="https://stackoverflow.com/questions/56008378/calling-other-functions-from-within-a-cuda-jit-numba-function">Calling other GPU Functions</a>

2. <a href="http://numba.pydata.org/numba-doc/dev/cuda/device-functions.html">Device Functions</a>



In [ ]:
import numpy as np
from time import perf_counter
from numba import cuda

@cuda.jit
def init_all(p_num, d_num, pos, accel, vel):
    for j in range(0, p_num[0]):
        for i in range(0, d_num[0]):
            pos[i,j] = 6.5
        # end for i
        for i in range(0, d_num[0]):
            accel[i,j] = 4.2*pos[i,j]
        # end for i
    # end for j

    init_vel(p_num, d_num, pos, accel, vel)
# end def

@cuda.jit(device=True)
def init_vel(p_num, d_num, pos, accel, vel):
    for j in range(0, p_num[0]):
        for i in range(0, d_num[0]):
            vel[i,j] = pos[i,i] + accel[i,j] * 0.1*pos[i,j]
        # end for i
    # end for j
# end def


# main loop
d_num = np.zeros(1, dtype=int)
p_num = np.zeros(1, dtype=int)
d_num[0] = 5000
p_num[0] = 5000
pos = np.zeros( shape=(d_num[0], p_num[0]), dtype=np.float32 )
accel = np.zeros( shape=(d_num[0], p_num[0]), dtype=np.float32 )
vel = np.zeros( shape=(d_num[0], p_num[0]), dtype=np.float32 ) 

start_time = perf_counter( )
d_p_num = cuda.to_device(p_num)
d_d_num = cuda.to_device(d_num)
d_pos = cuda.to_device(pos)
d_accel = cuda.to_device(accel)
d_vel = cuda.to_device(vel)

init_all(p_num, d_num, pos, accel, vel)

stop_time = perf_counter( )

print(pos)
print(vel)
print(accel)
print('')
print('    Elapsed wall clock time = %g seconds.' % (stop_time - start_time) )
print('')

[[6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 ...
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]
 [6.5 6.5 6.5 ... 6.5 6.5 6.5]]
[[24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 ...
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]
 [24.244999 24.244999 24.244999 ... 24.244999 24.244999 24.244999]]
[[27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 ...
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]
 [27.3 27.3 27.3 ... 27.3 27.3 27.3]]

    Elapsed wall clock time = 21.7524 seconds.



<a id='CUPY'></a>
## 4. CuPy

As discussed in the slides, CuPy is roughly a GPU equivalent for NumPy. It covers virtually
all of the NumPy functions but runs them on a GPU. In addition, CuPy is also starting to port
some SciPy routine to the GPU in a new library named <tt>cupyx.scipy</tt>.

The following examples illustrate how CuPy can be used in your Python code.

<a id='CUPY SVD Data on GPU'></a>
## 4.1 SVD Example, Leaving the Data on the GPU

This example performs a Singular Value Decomposition (SVD) on a random matrix that is
created on the GPU. Note that the results of the SVD, the <tt>u</tt>, <tt>v</tt>, and
<tt>s</tt> matrices, are available on the GPU after the computation but in this example,
they are not copied back to the host.

In [ ]:
import cupy as cp
import numpy as np
from time import perf_counter

size = 5000

start_time = perf_counter( )
A = np.random.uniform(low=-1., high=1., size=(size, size)).astype(np.float32)
u, s, v = np.linalg.svd(A)
stop_time = perf_counter( )

print('')
print('    Elapsed wall clock time for numpy = %g seconds.' % (stop_time - start_time) )
print('')


start_time = perf_counter( )
A = cp.random.uniform(low=-1., high=1., size=(size, size)).astype(cp.float32)
u, s, v = cp.linalg.svd(A)
stop_time = perf_counter( )

print('')
print('    Elapsed wall clock time for cupy = %g seconds.' % (stop_time - start_time) )
print('')


    Elapsed wall clock time for numpy = 83.3451 seconds.


    Elapsed wall clock time for cupy = 56.3107 seconds.



<a id='CUPY SVD Data back to host'></a>
## 4.2 SVD Example - Copy Data Back to the Host

This example is the same as the previous, but the <tt>u</tt> matrix is copied
back to the host using the <tt>asnumpy</tt> method. The "type" of the <tt>u</tt>
matrix on the GPU and the <tt>u</tt> matrix on the CPU are both printed so you
can tell that one is on the device (GPU) and one is on the host (CPU). It also
checks the difference between the reconstructed <tt>A</tt> matrix from the SVD
components, versus the original <tt>A</tt> matrix.

Reference: https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html

In [ ]:
import cupy as cp
import numpy as np

A_cpu = np.random.uniform(low=-1., high=1., size=(64, 64)).astype(np.float32)
A_gpu = cp.asarray(A_cpu)

u_gpu, s_gpu, v_gpu = cp.linalg.svd(A_gpu)
print("type(u_gpu) = ",type(u_gpu) )

u_cpu = cp.asnumpy(u_gpu)
print("type(u_cpu) = ",type(u_cpu) )

# ----- Check answer -----
v_cpu = cp.asnumpy(v_gpu)
s_cpu = cp.asnumpy(s_gpu)

s_cpu_test = np.diag(s_cpu)
result = np.allclose(A_cpu, np.dot(u_cpu, np.dot(s_cpu_test, v_cpu)), atol=1e-05)
print("Check result = ",result)

type(u_gpu) =  <class 'cupy.core.core.ndarray'>
type(u_cpu) =  <class 'numpy.ndarray'>
Check result =  True


<a id='CUPY Matrix Mult Data on GPU'></a>
## 4.3 Matrix Multiplication Example - Create Data on GPU

This example creates random data on the CPU or GPU and then performs the matrix
multiplication. Note that the result of the multiplication, the <tt>C</tt> matrix,
is available on the GPU if you need it.

Notice the similarity of the two parts of the code (numpy and cupy).
They are virtually identical.

In [ ]:
import math
import cupy as cp
import numpy as np
from time import perf_counter

size = 8000

start_time = perf_counter( )
A = np.random.uniform(low=-1.0, high=1.0, size=(size,size) ).astype(np.float32)
B = np.random.uniform(low=-1., high=1., size=(size,size) ).astype(np.float32)
C = np.matmul(A,B)
stop_time = perf_counter( )

print('')
print('    Elapsed wall clock time for numpy = %g seconds.' % (stop_time - start_time) )
print('')



start_time = perf_counter( )
A = cp.random.uniform(low=-1.0, high=1.0, size=(size,size) ).astype(cp.float32)
B = cp.random.uniform(low=-1., high=1., size=(size,size) ).astype(cp.float32)
C = cp.matmul(A,B)
stop_time = perf_counter( )

print('')
print('    Elapsed wall clock time for cupy = %g seconds.' % (stop_time - start_time) )
print('')


    Elapsed wall clock time for numpy = 13.1326 seconds.


    Elapsed wall clock time for cupy = 1.43566 seconds.



<a id='CUPY Matrix Mult Data on CPU'></a>
## 4.4 Matrix Multiplication Example - Copy Data from Host to GPU

This example creates the data on the CPU and then copies it to
the GPU. Matrix Multiplication on the CPU and GPU is timed. The
GPU timing includes the time for the data movement.

In [ ]:
import math
import cupy as cp
import numpy as np
from time import perf_counter

size = 8000

start_time = perf_counter( )
A = np.random.uniform(low=-1.0, high=1.0, size=(size,size) ).astype(np.float32)
B = np.random.uniform(low=-1., high=1., size=(size,size) ).astype(np.float32)
C = np.matmul(A,B)
stop_time = perf_counter( )

print('')
print('    Elapsed wall clock time for numpy = %g seconds.' % (stop_time - start_time) )
print('')

start_time = perf_counter( )
A_gpu = cp.asarray(A)
B_gpu = cp.asarray(B)
#A = cp.random.uniform(low=-1.0, high=1.0, size=(size,size) ).astype(cp.float32)
#B = cp.random.uniform(low=-1., high=1., size=(size,size) ).astype(cp.float32)
C_gpu = cp.matmul(A_gpu,B_gpu)
stop_time = perf_counter( )

print('')
print('    Elapsed wall clock time for cupy = %g seconds.' % (stop_time - start_time) )
print('')

C_cpu = cp.asnumpy(C_gpu)
result = np.allclose(C, C_cpu, atol=1e-03)
print("    Check result = ",result)


    Elapsed wall clock time for numpy = 13.0114 seconds.


    Elapsed wall clock time for cupy = 0.391918 seconds.

    Check result =  True


<a id='DASK example'></a>
# 5. Dask Example

Dask allows you to code different tasks to solve a problem and distribute
them to different nodes. It has been developed in coordinated manner with NumPy,
Pandas, and Scikit-learn, among others.

<a id='Dask create client'></a>
## 5.1 First Step - Create Dask Client

This example will illustrate how to compute an SVD of a matrix. We're going to
test it on a single GPU to learn how it works. The performance won't be good
since multiple tasks will share a single GPU.

The first step in using Dask is to create a Dask "client".

In [ ]:
from dask.distributed import Client

client = Client()

<a id='Dask serial SVD'></a>
## 5.2 Compute SVD using Numpy

For comparison, let's compute the Singular Value Decomposition (SVD) on
our CPU. The SVD is one of the algorithms that can be computed using Dask
(distributed computation). The code below is simple NumPy code to compute
the SVD of a random matrix.

In [ ]:
import numpy as np
from time import perf_counter

start_time = perf_counter( )
x = np.random.random((5000, 1000))

u, s, v = np.linalg.svd(x)
stop_time = perf_counter( )

print('')
print('    Elapsed wall clock time for CPU SVD = %g seconds.' % (stop_time - start_time) )
print('')

<a id='Dask parallel SVD task graph'></a>
## 5.3 Creating the Task Graph

One of the cool things about Dask is that you can use the classic NumPy functions!
The matrix is random and is created on the first CPU.

The code below illustrates how to use Dask to run the distributed SVD. Note that
you use the NumPy function for the SVD but Dask intercepts the call so that it
can use it's distributed routine. The function call, <tt>np.linalg.svd</tt>, actually performs
the SVD in a distributed manner. In this example, we're just running it on a single
node utilizing the CPUs.

Dask uses a graph algorithm for computing. The code below creates the task graph
for computing the SVD. The last command displays the task graph. It might seem a
bit complicated, but Dask hides all that complexity from you.

In [ ]:
import numpy as np
import dask.array as da
import dask

x = np.random.random((5000, 1000))

d = da.from_array(x, chunks=(1000, 1000))

u, s, v = np.linalg.svd(d)

dask.visualize(u, s, v)

<a id='Dask parallel SVD compute'></a>
## 5.4 Bringing the Results Back to a Single Node

Now let's compute the SVD!! In fact, let's compute the singular values
(<tt>s</tt> matrix), and the two unitary matrices, <tt>u</tt> and <tt>v</tt>.
After the computations, the firs singular value is printed as well as the
"shape" of the unitary matrices.

Finally, the code performs a "check" on the decomposition by comparing
it to the original matrix.

In [ ]:
from time import perf_counter

start_time = perf_counter( )
s_local = s.compute()
u_local = u.compute()
v_local = v.compute()
stop_time = perf_counter( )
print("First singular value = ",s_local[0])
print("shape(u) = ", u.shape)
print("shape(v) = ", v.shape)

print('')
print('Elapsed wall clock time for CPU SVD = %g seconds.' % (stop_time - start_time) )
print('')

# ----- Check answer -----
s_local_test = np.diag(s_local)
result = np.allclose(x, np.dot(u_local, np.dot(s_local_test, v_local)), atol=1e-04)
print("Check result = ",result)


The speed is likely to be slower than what we computed on the CPU but that's
not the point. The point is to show you how simple it is to use Dask to use
more resources to solve the SVD.

<a id='Dask parallel SVD CuPy'></a>
## 5.5 Run SVD using Dask and CuPy - Task Graph

We can do the same SVD but run it on GPUs! Dask is already GPU
aware and can interoperate with CuPy, so we really only need to create
the array on the GPUs (or copy to the GPUs) and tell Dask to distribute
across the nodes, and then compute!

You can run this simple example on your laptop or GPU VM if you have an
NVIDIA GPU, but remember that you only have one GPU, so the work
becomes serialzed and may take longer than computing across all of the
CPU cores. But it will compute the correct answer.

In [ ]:
import numpy as np
import cupy
import dask.array as da

x = cupy.random.random((5000, 1000))

d = da.from_array(x, chunks=(1000, 1000))

u, s, v = np.linalg.svd(d)

<a id='Dask parallel SVD CuPy compute'></a>
### 5.5.1 Compute the SVD
The <tt>compute()</tt> method performs the computations and brings the distributed
pieces of the computed results back to the starting node. At this point
they are local to this node.

The codebelow also "checks" the decomposition by comparing it to the
original matrix.

In [ ]:
from time import perf_counter

start_time = perf_counter( )
s_local = s.compute()
u_local = u.compute()
v_local = v.compute()
stop_time = perf_counter( )
print("First singular value = ",s_local[0])
print("shape(u) = ", u.shape)
print("shape(v) = ", v.shape)

print('')
print('Elapsed wall clock time for GPU SVD = %g seconds.' % (stop_time - start_time) )
print('')

# ----- Check answer -----
s_local_test = np.diag(s_local)
result = np.allclose(x, np.dot(u_local, np.dot(s_local_test, v_local)), atol=1e-04)
print("Check result = ",result)

<a id='Dask stop client'></a>
## 5.6 Need to Stop the Client

In [ ]:
from dask.distributed import Client
client.shutdown()

Appendix

Setup for colab environment:


In [8]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.23.1-he433501_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.4.16-py36_0 ...
installing: chardet-3.0.4-py36h0f667ec_1 ...
installing: idna-2.6-py36h82fb2a8_1 ...
installing: pycosat-0.6.3-py36h0a5515d_0 ...
installing: pycparser-2.18-py36hf9f622e_1 ...
installing: pysocks-1.6.8-py36_0 ...
installing: ruamel_yaml-0.15.37-py36h14c3975_2 ...
installing: six-1.11

--2021-02-26 14:52:15--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2021-02-26 14:52:15--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 7.31M 8s
    50K .......... .......... .......... .......... ..........  0%

In [9]:
%%bash
conda install --channel defaults conda python=3.6 --yes
conda update --channel defaults --all --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - conda
    - python=3.6


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    six-1.15.0                 |     pyhd3eb1b0_0          13 KB
    certifi-2020.12.5          |   py36h06a4308_0         144 KB
    wheel-0.36.2               |     pyhd3eb1b0_0          31 KB
    ca-certificates-2021.1.19  |       h06a4308_0         128 KB
    tqdm-4.56.0                |     pyhd3eb1b0_0          76 KB
    xz-5.2.5                   |       h7b6447c_0         438 KB
    tk-8.6.10                  |       hbc83047_0         3.2 MB
    python-3.6.12              |       hcff3b4d_2        34.0 MB
    _libgcc_mutex-0.1          |             main           3 KB
    pip-21.0.1                 |   py36h06a4308_0         2.0 MB
    pysocks-1.7.1              |   py36h06a4308_0   

six-1.15.0           |   13 KB | ########## | 100% 
certifi-2020.12.5    |  144 KB | ########## | 100% 
wheel-0.36.2         |   31 KB | ########## | 100% 
ca-certificates-2021 |  128 KB | ########## | 100% 
tqdm-4.56.0          |   76 KB | ########## | 100% 
xz-5.2.5             |  438 KB | ########## | 100% 
tk-8.6.10            |  3.2 MB | ########## | 100% 
python-3.6.12        | 34.0 MB | ########## | 100% 
_libgcc_mutex-0.1    |    3 KB | ########## | 100% 
pip-21.0.1           |  2.0 MB | ########## | 100% 
pysocks-1.7.1        |   30 KB | ########## | 100% 
conda-package-handli |  967 KB | ########## | 100% 
readline-8.1         |  464 KB | ########## | 100% 
yaml-0.2.5           |   87 KB | ########## | 100% 
libgcc-ng-9.1.0      |  8.1 MB | ########## | 100% 
libffi-3.3           |   54 KB | ########## | 100% 
ncurses-6.2          |  1.1 MB | ########## | 100% 
ld_impl_linux-64-2.3 |  645 KB | ########## | 100% 
zlib-1.2.11          |  120 KB | ########## | 100% 
chardet-4.0.

In [10]:
!conda --version

conda 4.9.2


In [11]:
import sys
_ = (sys.path.append("/usr/local/lib/python3.6/site-packages"))

In [12]:
!conda install --channel conda-forge cupy numba dask tbb --yes

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cupy
    - dask
    - numba
    - tbb


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bokeh-1.4.0                |   py36h9f0ad1d_1        13.5 MB  conda-forge
    ca-certificates-2020.12.5  |       ha878542_0         137 KB  conda-forge
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    click-7.1.2                |     pyh9f0ad1d_0          64 KB  conda-forge
    cloudpickle-1.6.0          |             py_0          22 KB  conda-forge
    conda-4.9.2                |   py36h5fab9bb_0         3.0 MB  conda-forge
    contextvars-2.4            |             py_0          11 KB  conda-fo

In [13]:
!conda list -e

# This file may be used to create an environment using:
# $ conda create --name <env> --file <this file>
# platform: linux-64
_libgcc_mutex=0.1=main
bokeh=1.4.0=py36h9f0ad1d_1
brotlipy=0.7.0=py36h27cfd23_1003
ca-certificates=2020.12.5=ha878542_0
certifi=2020.12.5=py36h5fab9bb_1
cffi=1.14.5=py36h261ae71_0
chardet=4.0.0=py36h06a4308_1003
click=7.1.2=pyh9f0ad1d_0
cloudpickle=1.6.0=py_0
conda=4.9.2=py36h5fab9bb_0
conda-package-handling=1.7.2=py36h03888b9_0
contextvars=2.4=py_0
cryptography=3.3.1=py36h3c74f83_1
cudatoolkit=10.2.89=hfd86e86_1
cudnn=7.6.5.32=h01f27c4_1
cupy=8.4.0=py36h99b894a_1
cytoolz=0.11.0=py36h1d69622_1
dask=2021.2.0=pyhd8ed1ab_0
dask-core=2021.2.0=pyhd8ed1ab_0
distributed=2021.2.0=py36h5fab9bb_0
fastrlock=0.5=py36h831f99a_1
freetype=2.10.4=h7ca028e_0
fsspec=0.8.7=pyhd8ed1ab_0
heapdict=1.0.1=py_0
idna=2.10=pyhd3eb1b0_0
immutables=0.14=py36h27cfd23_0
jinja2=2.11.3=pyh44b312d_0
jpeg=9d=h36c2ea0_0
ld_impl_linux-64=2.33.1=h53a641e_7
libblas=3.9.0=8_openblas
libcblas=3.9.0=8_o

In [14]:
!conda --version

conda 4.9.2


In [15]:
!python --version

Python 3.6.12 :: Anaconda, Inc.


In [16]:
!export NUMBA_THREADING_LAYER='omp'